<a href="https://colab.research.google.com/github/junyanzh/dataprocess-py/blob/main/natives_region.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [2]:
worksheet = gc.open('sme_orders').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

natives= pd.DataFrame.from_records(rows)
nativeshead = natives.iloc[0]
natives = natives[1:]
natives.columns = nativeshead
del nativeshead

In [3]:
worksheet = gc.open('natives_region_list').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

native_ls= pd.DataFrame.from_records(rows)
nativeshead = native_ls.iloc[0]
native_ls = native_ls[1:]
native_ls.columns = nativeshead
del nativeshead

In [4]:
native_ls

,山地原住民,平地原住民
1,烏來區,關西鎮
2,復興區,南庄鄉
3,五峰鄉,獅潭鄉
4,泰安鄉,魚池鄉
5,和平區,滿州鄉
6,仁愛鄉,壽豐鄉
7,阿里山鄉,光復鄉
8,那瑪夏區,豐濱鄉
9,三地門鄉,瑞穗鄉
10,大同鄉,富里鄉


In [5]:
native_region =pd.concat([native_ls['山地原住民'],native_ls['平地原住民']]).values.tolist()

In [6]:
natives[12320:12325]

,Order No,Sme Ban,Sme Comp Nm,Sme Address,Project Uuid,Project Nm,Project Price
12321,20211125131805929863,91192365,千堤北大髮妝工作室,新北市三峽區大德路１８８號,CA4618F7ED8C06A6E0531512620A6C3C,智能雲端美髮美膚數位服務-旗艦方案,"37,500"
12322,20211125131935165495,85029099,羅賓斯科技股份有限公司,臺北市松山區民權東路3段181號3樓,C158FD5741656B7BE0531512620A85F0,11-50人中小企業-Apollo雲端人資系統 (1+4)年，超值人才發展+員工福利方案,"40,000"
12323,20211125132458671562,17935932,玉露茶園,宜蘭縣大同鄉松羅村鹿場路１０之２號,CDCFD72101492F8EE0531512620A42FD,台灣好農國際館跨境商城開店方案,"37,500"
12324,20211125133615073153,85978063,大方早餐店,嘉義縣民雄鄉寮頂村民權路42之2號,C158FD573E286B7BE0531512620A85F0,專業版WPOS,"14,400"
12325,20211125133700575492,85978063,大方早餐店,嘉義縣民雄鄉寮頂村民權路42之2號,C158FD573E336B7BE0531512620A85F0,快一點LINE@點餐+掃碼點餐,"10,128"


In [7]:
import re
r = re.compile('(?P<zipcode>^\d{5}|^\d{3})?(?P<縣市>\D+?[縣市])?(?P<鄉鎮市區>\D+?[鄉鎮市區])?(?P<村里>\D+?[村里])?(?P<路街道段>\D+[路街道段])?(?P<鄰>\d+鄰)?(?P<巷>\d+巷)?(?P<弄>\d+弄)?(?P<號>\d+號)?(?P<樓>\d+樓)?(?P<其他>.+)?')

In [8]:
example = '新北市烏來區新烏路5段33號'
#'烏來區' in example

In [9]:
addressgroup=r.match(example)
addressgroup.groupdict().get('鄉鎮市區')

'烏來區'

In [10]:
region_df =pd.DataFrame()
for i in range(len(natives)):
  rg = r.match(natives['Sme Address'].iloc[i]).groupdict().get('鄉鎮市區')
  address = natives['Sme Address'].iloc[i]
  rg_dict={
      'region':rg
  }
  temp = pd.DataFrame(rg_dict,index=[i])
  region_df = region_df.append(temp,ignore_index=True)

In [11]:
natives.reset_index(inplace=True)

In [12]:
natives['region']=region_df

In [13]:
native_codintion = pd.DataFrame()
for i in range(len(natives)):
  if natives['region'].iloc[i] in native_region:
    nati= '原住民鄉鎮'
  else:
    nati= '非原住民鄉鎮'
  nati_dict={
      'native_codintion':nati
  }
  temp = pd.DataFrame(nati_dict,index=[i])
  native_codintion = native_codintion.append(temp,ignore_index=True)

In [14]:
natives['native_codintion'] =native_codintion
natives[10000:10050]

,index,Order No,Sme Ban,Sme Comp Nm,Sme Address,Project Uuid,Project Nm,Project Price,region,native_codintion
10000,10001,20211111123801939557,85011372,川字水國際事業股份有限公司,新北市淡水區新市一路3段101巷120號5樓,CDCFD720FCFC2F8EE0531512620A42FD,Ocard 會員集點管理系統 - 進階經營版(加購App曝光),"24,000",淡水區,非原住民鄉鎮
10001,10002,20211111123845890771,85041777,永怡德生物科技有限公司,臺南市仁德區仁德里文華路3段428巷150號1樓,CDCFD72101492F8EE0531512620A42FD,台灣好農國際館跨境商城開店方案,"37,500",仁德區,非原住民鄉鎮
10002,10003,20211111124649806818,97919224,典安大藥局,臺中市西屯區福林里西屯路三段１５９之６０號１樓,CDCFD720FD162F8EE0531512620A42FD,Ocard 線上點餐/訂購系統(加購App曝光),"15,000",西屯區,非原住民鄉鎮
10003,10004,20211111125842968361,88503891,良芳茶園焙茶工作室,臺中市龍井區東海里藝術街８６號１樓,C158FD57408D6B7BE0531512620A85F0,資廚- iCHEF POS 標準方案,"23,400",龍井區,非原住民鄉鎮
10004,10005,20211111130051331934,39764052,天富小吃店,彰化縣秀水鄉福安村中山路511號1樓,C158FD57436E6B7BE0531512620A85F0,【快一點】LINE點餐客戶服務系統,"37,500",秀水鄉,非原住民鄉鎮
10005,10006,20211111130101641596,81958684,禮好民宿,嘉義縣竹崎鄉竹崎村15鄰民生路４１之３號一樓,C158FD57408D6B7BE0531512620A85F0,資廚- iCHEF POS 標準方案,"23,400",竹崎鄉,非原住民鄉鎮
10006,10007,20211111130218911018,24312593,凡音文化有限公司,臺北市大安區仁愛路4段33號12樓之3,C158FD5740056B7BE0531512620A85F0,匯雲NexTrek-中小企業雲端記帳與金流管理工具,"11,988",大安區,非原住民鄉鎮
10007,10008,20211111130755807414,88003523,咖方弁輕食廚房,高雄市鳳山區鳳東路484號5樓,C158FD57436E6B7BE0531512620A85F0,【快一點】LINE點餐客戶服務系統,"37,500",鳳山區,非原住民鄉鎮
10008,10009,20211111130943857672,24312593,凡音文化有限公司,臺北市大安區仁愛路4段33號12樓之3,C2AAED66236D14CCE0531512620A1280,10人以下小企業-Apollo雲端人資系統 (1+4)年，超值人才發展+員工福利方案,"20,000",大安區,非原住民鄉鎮
10009,10010,20211111131529368306,87396084,凱興商行,桃園市桃園區中聖里吉昌街202號(1樓),C158FD57436E6B7BE0531512620A85F0,【快一點】LINE點餐客戶服務系統,"37,500",桃園區,非原住民鄉鎮
